In [1]:
import ee
ee.Initialize()

In [3]:
import pandas as pd
from io import StringIO
import urllib, io, os
from skimage import filters
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.patches as patches
import folium
import geehydro
from IPython.display import display
import ipywidgets as widgets
from ipywidgets import GridspecLayout, Layout

In [4]:
def AV (x):
    '''To interpolate Volume from Area '''
    for i in range (1,len(ARA)):
        x1=0
        x2=0
        y1=0
        y2=0
        if x>ARA[i]:
            pass
        else:
            x2=ARA[i]
            x1=ARA[i-1]
            y2=ARV[i]
            y1=ARV[i-1]
            break
    res=np.round(y1+((x-x1)*(y2-y1)/(x2-x1)),2)
    
#     print ("x:",x1,x2)
#     print ("y:",y1,y2)
    return res

def LV (x):
    '''To interpolate Volume from Level'''
    for i in range (1,len(ARL)):
        if x>ARL[i]:
            pass
        else:
            x2=ARL[i]
            x1=ARL[i-1]
            y2=ARV[i]
            y1=ARV[i-1]
            var = ((x-x1)*(y2-y1))/(x2-x1)
            res=np.round(y1+ var,2)
            
            break
#     print ("x:",x1,x2)
#     print ("y:",y1,y2)    
    return res


def LA (x):
    '''To interpolate area from Level'''
    for i in range (1,len(ARL)):
        if x>ARL[i]:
            pass
        else:
            x2=ARL[i]
            x1=ARL[i-1]
            y2=ARA[i]
            y1=ARA[i-1]
            var = ((x-x1)*(y2-y1))/(x2-x1)
            res=np.round(y1+ var,2)
            
            break
#     print ("x:",x1,x2)
#     print ("y:",y1,y2)
    return res

# trapezoidal formula
def simpson(A1,A2,H):
    '''trapezoidal formula'''
    return np.round(np.abs(H)*(A1+A2+np.sqrt(A1*A2))/3,2)


def clipping(image, geometry):
    return image.clip(geometry)


def estimateWSA(images,geometry,water_bands,water_threshold, water_sigma):
    geometry = ee.FeatureCollection(geometry)
    WSA_list = []
    global Dates_list
    Dates_list = []
    listOfImages = images.toList(images.size())

    Total_images = images.size().getInfo()
    
    global ndwi_list
    ndwi_list=[]
    for image_number in range(Total_images):

        image = ee.Image(listOfImages.get(image_number))
        ndwi = ee.Image(image).normalizedDifference(water_bands)
        
        ndwi_list.append(ndwi)
        
        edge = ee.Algorithms.CannyEdgeDetector(ndwi, water_threshold, water_sigma)
        ndwi_buffer = ndwi.mask(edge.focal_max(30, 'square', 'meters'))

        hist = ndwi_buffer.reduceRegion(ee.Reducer.histogram(150), geometry, 30).getInfo()
        values = ndwi_buffer.reduceRegion(ee.Reducer.toList(), geometry, 30).getInfo()

        th = filters.threshold_otsu(np.array(values['nd']))
        water = ndwi.gt(th)

        water_edge = ee.Algorithms.CannyEdgeDetector(water, 0.5, 0)

        area = ee.Image.pixelArea();
        waterArea = water.multiply(area).rename('waterArea');

        image = image.addBands(waterArea);

        stats = waterArea.reduceRegion(reducer=ee.Reducer.sum(),geometry= geometry,scale=30,crs = image.select('waterArea').projection())
        image_details = image.getInfo()

        WSA = stats.getInfo()

        Dates_list.append(image_details['properties']['DATE_ACQUIRED'])
        WSA_list.append(round((WSA['waterArea']/10**4),2))
        
    return Dates_list,WSA_list

In [7]:
water_sigma = 1
water_threshold = 0.5
water_bands = ['B3', 'B6']


def estimateRS(image_collection,geometry,start,stop,spillway_storage,path,row,cloud_cover,reservoir_data,water_level_data,sedimentation_duration,name=None):
                
        geometry = ee.FeatureCollection(geometry)

        global shapefile
        shapefile = geometry
        

        # generate percentile composite image
        images = ee.ImageCollection(image_collection)\
        .filterBounds(geometry)\
        .filter(ee.Filter.eq('WRS_PATH', path))\
        .filter(ee.Filter.eq('WRS_ROW', row))\
        .filterDate(start,stop).filterMetadata('CLOUD_COVER', 'less_than', cloud_cover)\
        #.map(clipping);

        dates,area = estimateWSA(images,geometry,water_bands,water_threshold,water_sigma)

        rs_area = pd.DataFrame(area,dates)
        rs_area.columns = ['waterArea']
        rs_area = rs_area/100 # convert to Mm2
        
        dam_df = pd.read_table(reservoir_data,sep=',',index_col=None)
        observed_df = pd.read_table(water_level_data,sep=',',index_col=0,parse_dates=True)
        
        global ARL, ARA, ARV
        ARL=dam_df['Level']
        ARA=dam_df['Area']
        ARV=dam_df['Capacity']
        
                       
        sort_df = {}
        for day in rs_area.index:
            if day in observed_df.index:
                sort_df[day]=np.float(observed_df.loc[day]['Level(m.)'])

        
        sort_df = pd.DataFrame(sort_df,index=['Level']).T
        sort_df['Area estimated'] = np.round(rs_area['waterArea'],2)

        # filtering wrong WSA estimation
        sorted_df1 = sort_df.sort_values(['Level','Area estimated'])
        sorted_df1['Original area']= sorted_df1['Level'].apply(LA)
        sorted_df1 = sorted_df1[sorted_df1['Area estimated']<sorted_df1['Original area']]
        
        sorted_df = pd.DataFrame(np.maximum.accumulate(sorted_df1['Area estimated']))
        sorted_df['Level'] = sorted_df1['Level']
        sorted_df = sorted_df.drop_duplicates(subset='Area estimated', keep="first")

        sorted_df['Original area']= sorted_df['Level'].apply(LA)
        
        #sorted_df['Original_capacity'] = sorted_df['Level'].apply(LV)- LV(sorted_df['Level'][0])
        
        estimated_capacity = np.zeros(len(sorted_df))
        original_capacity2 = np.zeros(len(sorted_df))
        for r in range(1,len(sorted_df)):
            A1 = sorted_df['Area estimated'][r-1]
            A2 = sorted_df['Area estimated'][r]
            H = sorted_df['Level'][r-1]-sorted_df['Level'][r]

            estimated_capacity[r]=simpson(A1,A2,H)

            Ao1 = sorted_df['Original area'][r-1]
            Ao2 = sorted_df['Original area'][r]
            original_capacity2[r]=simpson(Ao1,Ao2,H)
        
        sorted_df['Estimated capacity']= estimated_capacity
        sorted_df['Original capacity']= original_capacity2
        
        sorted_df['Cumulative estimated capacity']= sorted_df['Estimated capacity'].cumsum()
        sorted_df['Cumulative original capacity']= sorted_df['Original capacity'].cumsum()
        
        
        display(sorted_df)
                
        
        #silt deposited b/w two consecutive surveys Mm3
        silt_deposited = np.round(sorted_df['Cumulative original capacity'][-1]-sorted_df['Cumulative estimated capacity'][-1],2)
        print("Reservoir capacity lost(MCM): "+str(silt_deposited))

        #rate of silt deposited b/w two consecutive surveys Mm3/year
        Periods_btw_surveys = sedimentation_duration
        rate_of_silt_deposited = np.round(silt_deposited/Periods_btw_surveys,2)
        print("Rate of siltation(MCM/yr): "+ str(rate_of_silt_deposited))

        # Life of reservoir
        total_capacity = spillway_storage # upto spillway crest
        life_of_reservoir = np.round(total_capacity/rate_of_silt_deposited,2)
        print("Life of reservoir(yrs): "+str(life_of_reservoir))


        #Revised Elevation-Capcity curves
        fig,ax = plt.subplots(figsize=(8,5))
        ax.plot(sorted_df['Level'],sorted_df[['Cumulative original capacity','Cumulative estimated capacity']])
        ax.legend(['Original capacity','Estimated capacity']);
        ax.set_xlabel('Level (m)')
        ax.set_ylabel('Capacity (MCM)')
        ax.set_title('Revised Elevation-Capacity curves');
        
        if name!=None:
            plt.savefig("Revised curves_"+str(name)+".jpg", dpi=300, bbox_inches='tight')
            #To download file
            if Download:
                sorted_df.to_csv('Sedimentation_analysis_'+str(name)+'.csv', header=True, index=True)
        else:
            plt.savefig("Revised curves.jpg", dpi=300, bbox_inches='tight')
            #To download file
            if Download:
                sorted_df.to_csv('Sedimentation_analysis.csv', header=True, index=True)
                
        display(fig)

<h1>RSEQuick</h1>
<h2>Fast reservoir sedimenatation assessment</h2>

In [10]:
geometry = widgets.Text(
    value='users/GEE_codes/Tenughat_reservoir_buffer',
    placeholder='GEE shapefile asset',
    description='Geometry:',
    disabled=False,
    layout=Layout(height='auto', width='auto')
)

image_collections = widgets.Dropdown(
    options=["LANDSAT/LC08/C01/T1_TOA",
    "LANDSAT/LE07/C01/T1_RT_TOA",
    "LANDSAT/LT05/C01/T2_TOA",
    "LANDSAT/LT05/C01/T1_TOA"],
    value="LANDSAT/LC08/C01/T1_TOA",
    description='Satellite:',
    disabled=False,
    layout=Layout(height='auto', width='auto'),
)

Start = widgets.DatePicker(
    description='Start Date',
    disabled=False,
    layout=Layout(height='auto', width='auto')
)

End = widgets.DatePicker(
    description='End Date',
    disabled=False,
    layout=Layout(height='auto', width='auto')
)

path = widgets.IntText(
    value=140,
    description='Path:',
    disabled=False,
    layout=Layout(height='auto', width='auto')
)

row = widgets.IntText(
    value=44,
    description='Row:',
    disabled=False,
    layout=Layout(height='auto', width='auto')
)


cloud_cover = widgets.IntSlider(
    description = "Cloud cover",
    value=5,
    layout=Layout(height='auto', width='auto')
    )

# Final_storage = widgets.BoundedFloatText(
#     value=649,
#     min = 0,
#     max = 1000,
#     step=1,
#     description='Total storage:',
#     disabled=False,
#     layout=Layout(height='auto', width='auto')
# )

spillway_storage = widgets.BoundedFloatText(
    value=347,
    min=0,
    max=1000,
    step=1,
    description='Active storage',
    disabled=False,
    layout=Layout(height='auto', width='auto')
)

reservoir_characteristics  = widgets.FileUpload(
    description = "Elevation Area Volume data",
    accept='.csv',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False,  # True to accept multiple files upload else False
    layout=Layout(height='auto', width='auto')
)

observed_data = widgets.FileUpload(
    description = "Observed Water Level (m)",
    accept='.csv',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False,  # True to accept multiple files upload else False
    layout=Layout(height='auto', width='auto')
)

Download = widgets.Checkbox(description = "Download Report",layout=Layout(height='auto', width='auto'))

Estimate_button = widgets.Button(
    description='Estimate',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Estimate reservoir sedimentaation',
    icon='check', # (FontAwesome names without the `fa-` prefix)
    layout=Layout(height='auto', width='auto')
)

name = widgets.Text(
    value=None,
    placeholder='Reservoir name',
    description='Name:',
    disabled=False,
    layout=Layout(height='auto', width='auto')
)

duration = widgets.IntSlider(
    description = "Duration:",
    value=6,
    layout=Layout(height='auto', width='auto')
    )

grid = GridspecLayout(7, 2)

items = [image_collections,
geometry,
Start,
End,
row,
path,
cloud_cover,
name,
spillway_storage,
duration,
reservoir_characteristics,
observed_data,
Download,
Estimate_button,]

item_no = 0
for x in range(8):
    for y in range(2):
        try:
            grid[x,y] = items[item_no]
        except:
            pass
        item_no+=1
        
# 
output = widgets.Output(layout={'border': '1px solid black'})
display(grid)


def on_button_clicked(b):
    output.clear_output()
    with output:
        print("Running...")
        geometryV = geometry.value
        image_collectionsV = image_collections.value
        StartV = str(Start.value)
        EndV = str(End.value)
        pathV = path.value
        rowV = row.value
        cloud_coverV  = cloud_cover.value
        #Final_storageV = Final_storage.value
        spillway_storageV   = spillway_storage.value
        datatbl1= StringIO(str(reservoir_characteristics.data[0],'utf-8'))
        datatbl2 = StringIO(str(observed_data.data[0],'utf-8'))
        nameV = name.value 
        durationV =duration.value
        
        estimateRS(image_collectionsV,geometryV,StartV,EndV,\
                   spillway_storageV,\
                   pathV,rowV,cloud_coverV,\
                   datatbl1,datatbl2,durationV,nameV)
        
        Map = folium.Map(location=[23.7175,85.8238], zoom_start=12,width="100%",height="100%")
        # To see a google satellite view as a basemap
        Map.setOptions('HYBRID')
        ndwiParams = {'min': -1, 'max': 1, 'palette': ['green', 'white', 'blue']}
        
        for num,img in enumerate(ndwi_list):
            Map.addLayer(img.clip(shapefile), ndwiParams, 'NDWI '+str(Dates_list[num]))

        Map.setControlVisibility(layerControl=True, fullscreenControl=False, latLngPopup=True)
        print("Finished")
        display(Map)
        
Estimate_button.on_click(on_button_clicked)

display(output)

GridspecLayout(children=(Dropdown(description='Satellite:', layout=Layout(grid_area='widget001', height='auto'…

Output(layout=Layout(border='1px solid black'))